In [1]:
path = './ntust-ir-2020_hw5_new/'
# path = '../HW2-BM25/ntust-ir-2020/'
import numpy as np
import numba as nb
import random
import time
import math
from tqdm import tqdm
from math import log
from collections import Counter, namedtuple
from numba.typed import List

In [2]:
# data of doc and query
bm_doc = []
query = []
# length of each doc
bm_doc_len = []
# Name of doc and query
bm_doc_list = []
query_list = []
# TF of doc and query
tf_doc = []
tf_q = []
# BM25 query vocabulary
bm25_vocabulary = set()
# BackGround voc
bg_voc = Counter()
# -----------------------------------------------------------------------------
# get doc files and query files
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp_query = f_temp.read().split()
        tf_q.append(Counter(temp_query))
        for w in temp_query: bm25_vocabulary.add(w)
        query.append(temp_query)
        f_temp.close()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        bm_doc_list.append(l)
        f_temp = open(path + 'docs/{}.txt'.format(l))
        temp_doc = f_temp.read().split()
        tf_doc.append(Counter(temp_doc))
        bg_voc.update(temp_doc)
        bm_doc.append(temp_doc)
        bm_doc_len.append(len(temp_doc))
        f_temp.close()

100%|██████████| 30000/30000 [00:06<00:00, 4558.47it/s]


In [3]:
bm25_vocabulary = list(bm25_vocabulary)
# All doc, query and vocabulary length
bm_d_len = len(bm_doc)
q_len = len(query)
bm_w_len = len(bm25_vocabulary)

In [4]:
# Build TF_key and TF_val
TF_doc_k = List()
TF_doc_v = List()
for d in tqdm(range(bm_d_len)):
    TF_doc_k.append(List([k for k in tf_doc[d].keys()]))
    TF_doc_v.append(List([v for v in tf_doc[d].values()]))
TF_q_k = List()
TF_q_v = List()
for q in range(q_len):
    TF_q_k.append(List([k for k in tf_q[q].keys()]))
    TF_q_v.append(List([v for v in tf_q[q].values()]))

100%|██████████| 30000/30000 [00:28<00:00, 1037.60it/s]


In [5]:
# BG voc handling
bg_vocabulary = []
for k,v in bg_voc.items():
    if(v > 200):
        bg_vocabulary.append(k)
len(bg_vocabulary)

5559

In [6]:
# BM25 IDF
@nb.njit
def bm_IDF(idf, tf_key, bm25_voc, dl, bm25l):
    for d in range(dl):
        for v in range(bm25l):
            if(bm25_voc[v] in tf_key[d]):
                idf[v] += 1 
    return idf

In [7]:
# BM25 IDF
bm_idf = np.zeros(len(bm25_vocabulary))
time_idf = time.time()
bm_idf = bm_IDF(bm_idf, TF_doc_k, bm25_vocabulary, bm_d_len, bm_w_len)
print('time: {:.5f} mins\n'.format((time.time() - time_idf) / 60))

time: 1.24550 mins



In [8]:
# B25 Model
@nb.njit
def BM25(sim, k1, b_, N_, avg, dl_list, dl, ql, tf_key, tf_val, tf_q_key, idf_, bm_voc):
    for q in range(ql):
        for d in range(dl):
            sum = 0.0
            for w in tf_q_key[q]:
                sum_tmp = 1.0
                if(w in tf_key[d]):
                    w_tf = tf_val[d][tf_key[d].index(w)]
                    ni_loc = bm_voc.index(w)
                    ni = idf_[ni_loc]
                    sum_tmp = (k1 + 1) * w_tf / (k1 * ((1 - b) + b * dl_list[d] / avg) + w_tf)
                    sum_tmp *= log((N_ - ni + 0.5) / (ni + 0.5))
                sum += sum_tmp
            sim[q][d] = sum
    return sim

In [9]:
# BM25 model variable
bm_sim = np.zeros([q_len, bm_d_len], dtype=float)
K1 = 0.8
b = 0.7
N = bm_d_len
avg_dl = np.average(bm_doc_len)

In [10]:
time_bm = time.time()
bm_sim = BM25(bm_sim, K1, b, N, avg_dl, bm_doc_len, bm_d_len, q_len, TF_doc_k, TF_doc_v, TF_q_k, bm_idf, bm25_vocabulary)
print('time: {:.2f} mins\n'.format((time.time() - time_bm) / 60))

time: 1.90 mins



In [11]:
# Get relevance document
Rele_len = int(6000)
rel_set = set()
Rq = np.zeros([q_len, Rele_len], dtype=np.int64)
for q in tqdm(range(q_len)):
    arg = np.argsort(-bm_sim[q])[:Rele_len]
    for r in range(Rele_len):
        Rq[q][r] = arg[r]
        rel_set.add(arg[r])

100%|██████████| 150/150 [00:00<00:00, 213.86it/s]


In [12]:
# relevance doc vocabulary lower
@nb.njit
def RD_v(result, rel_doc, rel_dl, tf_doc_k, tf_doc_v):
    not_valid = []
    for d in range(rel_dl):
        for w in tf_doc_k[rel_doc[d]]:
            if(w not in result):
                s = 0
                for k in range(rel_dl):
                    if(w in tf_doc_k[k]):
                        loc = tf_doc_k[k].index(w)
                        s += tf_doc_v[k][loc]
                if(s > 300):
                    result.append(w)
                else:
                    not_valid.append(w)
    return result

In [13]:
# VSM for each query
def VSM(q_sim, q_, rel_doc, rel_dl, ql, tf_doc_k, tf_doc_v, tf_q_k, tf_q_v, Voc):
    voc = Voc
    # make vocabulary
    for q in q_:
        if(q not in voc):
            voc.append(q)
    # # 1/5 relevance doc
    # ratio = int(rel_dl / 4)
    # for d in tf_doc_k[:ratio]:
    #     for w in d:
    #         voc.add(w)
    # voc = list(voc)
    v_len = len(voc) # N = v_len
    # query
    q_tfidf = np.zeros(v_len)
    # query idf
    @nb.njit
    def Q_idf(idf_, voc_, query_, q_v, q_l, v_l):
        for q in range(q_l):
            loc = voc_.index(query_[q])
            idf_[loc] = log(v_l / q_v[q])
        return idf_
    q_time = time.time()
    q_tfidf = Q_idf(q_tfidf, voc, q_, tf_q_v, ql, v_len)
    # query tf-idf
    @nb.njit
    def Q_tfidf(tfidf, voc_, query_, q_v, q_l, v_l):
        for v in range(v_l):
            if(tfidf[v] == 0): tfidf[v] = log(v_l / 3)
            for q in range(q_l):
                loc = voc_.index(query_[q])
                tfidf[loc] *= (0.5 + 0.5 * q_v[q])
        return tfidf
    q_tfidf = Q_tfidf(q_tfidf, voc, q_, tf_q_v, ql, v_len)
    print("\nQ_time: {:.3f}\n".format((time.time() - q_time) / 60))
    # Doc
    @nb.njit
    def D_idf(idf_, voc_, rel_, rel_l, doc_k, v_l):
        for v in range(v_l):
            for d in range(rel_l):
                if(voc_[v] in doc_k[rel_[d]]):
                    idf_[v] += 1
            if(idf_[v] == 0):
                idf_[v] = 1
            else:
                idf_[v] = log(v_l / idf_[v])
        return idf_
    # doc_idf
    d_idf = np.zeros(v_len)
    d_time = time.time()
    d_idf = D_idf(d_idf, voc, rel_doc, rel_dl, tf_doc_k, v_len)
    print("D_time: {:.3f}\n".format((time.time() - d_time) / 60))
    # Cos_sim
    @nb.njit
    def cos_sim(sim, q_tf_idf, D_idf, rel_, rel_l, doc_k, doc_v, q_l, voc_):
        for d in range(rel_l):
            cp_doc_tfidf = np.copy(D_idf)
            l = len(doc_k[rel_[d]])
            for w in range(l):
                if(doc_k[rel_[d]][w] in voc_):
                    loc = voc_.index(doc_k[rel_[d]][w])
                    cp_doc_tfidf[loc] *= doc_v[rel_[d]][w]
            # cos sim cal
            mol = np.dot(q_tf_idf, cp_doc_tfidf)
            den = math.sqrt(np.sum(q_tf_idf**2)) * math.sqrt(np.sum(cp_doc_tfidf**2))
            sim[d] = mol / den
        return sim
    sim_time = time.time()
    q_sim = cos_sim(q_sim, q_tfidf, d_idf, rel_doc, rel_dl, tf_doc_k, tf_doc_v, ql, voc)
    print("\nSim_time: {:.3f}\n".format((time.time() - sim_time) / 60))
    return q_sim

In [14]:
vsm_sim = np.zeros([q_len, Rele_len])
for q in tqdm(range(q_len)):
    print("\nIteration {}:\n".format(q+1))
    vsm_sim[q] = VSM(vsm_sim[q], query[q], Rq[q], Rele_len, len(query[q]), TF_doc_k, TF_doc_v, TF_q_k[q], TF_q_v[q], bg_vocabulary)

  0%|          | 0/150 [00:00<?, ?it/s]
Iteration 1:


Q_time: 0.014

D_time: 7.958

  1%|          | 1/150 [11:49<29:21:45, 709.43s/it]
Sim_time: 3.850


Iteration 2:


Q_time: 0.058

D_time: 6.676

  1%|▏         | 2/150 [22:01<27:58:07, 680.32s/it]
Sim_time: 3.473


Iteration 3:


Q_time: 0.048

D_time: 7.521

  2%|▏         | 3/150 [33:30<27:52:45, 682.76s/it]
Sim_time: 3.905


Iteration 4:


Q_time: 0.051

D_time: 5.924

  3%|▎         | 4/150 [42:31<25:58:22, 640.43s/it]
Sim_time: 3.052


Iteration 5:


Q_time: 0.014

D_time: 7.813

  3%|▎         | 5/150 [54:23<26:39:33, 661.88s/it]
Sim_time: 4.039


Iteration 6:


Q_time: 0.023

D_time: 7.311

  4%|▍         | 6/150 [1:05:36<26:36:04, 665.03s/it]
Sim_time: 3.872


Iteration 7:


Q_time: 0.023

D_time: 6.728

  5%|▍         | 7/150 [1:15:41<25:42:26, 647.18s/it]
Sim_time: 3.341


Iteration 8:


Q_time: 0.049

D_time: 5.465

  5%|▌         | 8/150 [1:24:06<23:50:34, 604.47s/it]
Sim_time: 2.900


Iteration 9:


Q_time: 0.037

D_ti

In [17]:
vsm_sim

array([[0.93998565, 0.92173952, 0.90827638, ..., 0.95246249, 0.94931155,
        0.95309828],
       [0.93916726, 0.95104292, 0.95217288, ..., 0.95453515, 0.95462582,
        0.93746073],
       [0.92205272, 0.94679991, 0.9474615 , ..., 0.95055036, 0.9502649 ,
        0.95041059],
       ...,
       [0.94317678, 0.95322091, 0.94480747, ..., 0.95415278, 0.94641203,
        0.91944619],
       [0.89894513, 0.89180497, 0.90817081, ..., 0.94866325, 0.94599534,
        0.93193379],
       [0.93236768, 0.92604933, 0.94429199, ..., 0.9439239 , 0.94620236,
        0.8304345 ]])

In [18]:
# Create output file
fp = open("roicchio.txt", "w")
fp.write("Query,RetrievedDocuments\n")
for i in tqdm(range(q_len)):
    fp.write('{},'.format(query_list[i]))
    for s in np.argsort(vsm_sim[i][:5000]):
        fp.write(bm_doc_list[Rq[i][s]] + ' ')
    fp.write('\n')
fp.close()

100%|██████████| 150/150 [00:00<00:00, 203.02it/s]
